In [15]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
from FPN import Features, FPNetwork , classificationhead , bboxhead
from Loss import Lossfunction
from datasets import load_dataset
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler
import gc
from dataset_convert import AnchorGenerator, FaceDetectionDataset
# device = torch.device("mps")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
model = model.features.to(device)

Using cache found in /home/.cache/torch/hub/pytorch_vision_v0.10.0
/home/faces2.0/.conda/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/faces2.0/.conda/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [17]:
dataset = load_dataset("CUHK-CSE/wider_face")
train_dataset = dataset['train'].with_format("torch")
val_dataset = dataset['validation'].with_format("torch")

In [18]:
extractor = Features(model,['3','6', '13','18'])
topdown = FPNetwork(out_channels=256)
classifier = classificationhead(channels=256, num_anchors= 12, num_of_classes= 1)
bboxregression = bboxhead(channels= 256 , num_anchors= 12)
loss =Lossfunction()
anchors = AnchorGenerator()
data = FaceDetectionDataset(train_dataset,anchors)
vali_data = FaceDetectionDataset(val_dataset,anchors)

In [19]:
# hyperparametrs
batch_size = 8
epochs = 50
learning_rate = 5e-4

In [ ]:
training_data = DataLoader(
    data,
    batch_size=8,           # Reduce if GPU memory is full
    num_workers=4,           # Use 4-8 workers for CPU-bound tasks
    # pin_memory=True,         # Faster CPU->GPU transfer
    persistent_workers=True, # Reuse workers across epochs
    prefetch_factor=4,       # Prefetch more batches
    drop_last=True,
    shuffle=True
)
# training_data = DataLoader(
#     data,
#     batch_size=8,           # Reduce if GPU memory is full
#     num_workers=4,           # Use 4-8 workers for CPU-bound tasks
#     # pin_memory=True,         # Faster CPU->GPU transfer
#     persistent_workers=True, # Reuse workers across epochs
#     prefetch_factor=4,       # Prefetch more batches
#     drop_last=True,
#     shuffle=True
# )

In [20]:
validation_data = DataLoader(vali_data, batch_size=batch_size, shuffle = True)

In [21]:
torch.cuda.empty_cache()

In [22]:
print(f"Allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"Cached: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

Allocated: 0.04 GB
Cached: 0.05 GB


In [23]:
# batch_size = 16
# epochs = 20
# learning_rate = 1e-3
def forward(p):
    features = extractor.extract(p)
    newfeatures = topdown(features)
    output = {}
    for key in list(newfeatures.keys()):
        temp = {}
        temp["bbox"] = bboxregression(newfeatures[key])
        temp["cls"] = classifier(newfeatures[key])
        output[key] = temp
    return output

def train(epochs:int,training_data , validation_data=None):
    extractor.train()
    topdown.train()
    classifier.train()
    bboxregression.train()
    optimizer = optim.Adam(list(extractor.parameters())+
                           list(topdown.parameters())+
                           list(classifier.parameters())+
                           list(bboxregression.parameters()), lr=learning_rate)
    loSS = {}
    for i in range (0,epochs):
        train_loss= 0
        for key, (image,bbox) in enumerate(training_data):
            epoch_loss = {}
            model_pred = forward(image)
            optimizer.zero_grad()
            ll = loss(model_pred, bbox)
            train_loss+=ll
            if (key % 10 == 0):
                epoch_loss[key] = train_loss/10
                print(f"The avg loss for {key}th data is {train_loss/10}")
                train_loss = 0    
            ll.backward()
            optimizer.step()
        loSS[i] = epoch_loss
    return loSS

# train(epochs=epochs,training_data=training_data)


In [24]:
total_params=sum(p.numel() for p in extractor.parameters())
total_params+= sum(p.numel() for p in topdown.parameters())
total_params+= sum(p.numel() for p in classifier.parameters())
total_params+= sum(p.numel() for p in bboxregression.parameters())

print(total_params)

7452536


In [25]:
# train(epochs=epochs,training_data=training_data)

In [26]:
scaler = GradScaler()

/tmp/ipykernel_7015/4292142752.py:1: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [27]:
def train_with_accumulation(epochs: int, training_data, validation_data=None, accumulation_steps=4):
    # Set models to training mode
    extractor.train()
    topdown.train()
    classifier.train()
    bboxregression.train()
    
    # Enable gradient checkpointing
    if hasattr(extractor, 'gradient_checkpointing_enable'):
        extractor.gradient_checkpointing_enable()
    if hasattr(topdown, 'gradient_checkpointing_enable'):
        topdown.gradient_checkpointing_enable()
    
    optimizer = optim.Adam(
        list(extractor.parameters()) +
        list(topdown.parameters()) +
        list(classifier.parameters()) +
        list(bboxregression.parameters()), 
        lr=learning_rate
    )
    
    loss_history = {}
    
    for epoch in range(epochs):
        epoch_loss = 0.0
        batch_count = 0
        running_loss = 0.0
        
        print(f"\nEpoch {epoch + 1}/{epochs}")
        print("-" * 30)
        
        for batch_idx, (image, bbox) in enumerate(training_data):
            # Mixed precision forward pass
            with autocast():
                model_pred = forward(image)
                ll = loss(model_pred, bbox) / accumulation_steps  # Scale loss
            
            # Backward pass
            scaler.scale(ll).backward()
            
            # Step optimizer every accumulation_steps
            if (batch_idx + 1) % accumulation_steps == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
            
            # Accumulate loss
            batch_loss = ll.item() * accumulation_steps  # Unscale for logging
            epoch_loss += batch_loss
            running_loss += batch_loss
            batch_count += 1
            
            # Print progress
            if (batch_idx + 1) % 10 == 0:
                avg_running_loss = running_loss / 10
                print(f"Batch {batch_idx + 1}: Avg Loss = {avg_running_loss:.6f}")
                running_loss = 0.0
            
            # Memory cleanup
            del model_pred, ll
            if batch_idx % 20 == 0:
                torch.cuda.empty_cache()
            break
        # Handle any remaining gradients
        if batch_count % accumulation_steps != 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
        
        avg_epoch_loss = epoch_loss / batch_count if batch_count > 0 else 0
        loss_history[epoch] = avg_epoch_loss
        
        print(f"Epoch {epoch + 1} Average Loss: {avg_epoch_loss:.6f}")
        
        # Cleanup at epoch end
        gc.collect()
        torch.cuda.empty_cache()
    
    return loss_history


In [28]:
loss_data = train_with_accumulation(epochs=epochs, training_data=training_data)#, validation_data= validation_data)



Epoch 1/50
------------------------------


/tmp/ipykernel_7015/1942544810.py:34: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1 Average Loss: 0.397116

Epoch 2/50
------------------------------
Epoch 2 Average Loss: 0.316766

Epoch 3/50
------------------------------
Epoch 3 Average Loss: 0.262612

Epoch 4/50
------------------------------
Epoch 4 Average Loss: 0.253129

Epoch 5/50
------------------------------
Epoch 5 Average Loss: 0.252356

Epoch 6/50
------------------------------
Epoch 6 Average Loss: 0.245461

Epoch 7/50
------------------------------
Epoch 7 Average Loss: 0.236408

Epoch 8/50
------------------------------
Epoch 8 Average Loss: 0.232772

Epoch 9/50
------------------------------
Epoch 9 Average Loss: 0.229413

Epoch 10/50
------------------------------
Epoch 10 Average Loss: 0.230995

Epoch 11/50
------------------------------
Epoch 11 Average Loss: 0.231634

Epoch 12/50
------------------------------
Epoch 12 Average Loss: 0.227808

Epoch 13/50
------------------------------
Epoch 13 Average Loss: 0.229457

Epoch 14/50
------------------------------
Epoch 14 Average Loss: 0.2256

In [31]:
torch.save(extractor.state_dict(),'/home/faces2.0/models/extractor.pt')
torch.save(topdown.state_dict(),'/home/faces2.0/models/topdown.pt')
torch.save(classifier.state_dict(),'/home/faces2.0/models/classifier.pt')
torch.save(bboxregression.state_dict(),'/home/faces2.0/models/bboxregression.pt')

